In [ ]:
! pip install timm==0.4.5
! pip install wget

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import argparse
import os
import ast
import pickle
import sys
import time
import torch
import numpy as np
import pandas as pd
import json

from sklearn.model_selection import train_test_split
from torch.utils.data import WeightedRandomSampler
basepath = os.path.dirname(os.path.dirname(sys.path[0]))
sys.path.append(basepath)

import dataloader
from ast_models import ASTModel
from traintest import train, validate

File Operations

In [ ]:
! wget "https://www.dropbox.com/s/36yqmymkva2bwdi/spcup_2022_training_part1.zip?dl=1" -c -O 'spcup_2022_training_part1.zip'
! wget "https://www.dropbox.com/s/wsmlthhri29fb79/spcup_2022_unseen.zip?dl=1" -c -O 'spcup_2022_unseen.zip'

In [ ]:
!unzip "./spcup_2022_training_part1.zip" -d "./spcup_2022_training/"
!unzip "./spcup_2022_unseen.zip" -d "./spcup_2022_unseen/"

In [6]:
!rm "./spcup_2022_training_part1.zip"
!rm "./spcup_2022_unseen.zip"

In [7]:
# create combined label df
# combine two label sets
df1 = pd.read_csv('./spcup_2022_training/spcup_2022_training_part1/labels.csv')
df2 = pd.read_csv('./spcup_2022_unseen/spcup_2022_unseen/labels.csv')
df3 = pd.concat([df1, df2]).sample(frac=1)

df3.to_csv('./final_labels.csv', index=False)
!rm './spcup_2022_unseen/spcup_2022_unseen/labels.csv'

In [8]:
# copy .wav files from unseen to all
!cp -a "./spcup_2022_unseen/spcup_2022_unseen/". "./spcup_2022_training/spcup_2022_training_part1/"

JSON creation

In [9]:
DATA_PATH = "./spcup_2022_training/spcup_2022_training_part1/"
IMAGE_PATH = './images'
Labeled_dir = './final_labels.csv'

In [10]:
label_df = pd.read_csv(Labeled_dir)
label_df.head()

,track,algorithm
0,d5142a1e904db7ef51b8071585c46c68.wav,4
1,0520f2abfaa77784411e0121ba3cb23d.wav,3
2,64fc1bd819a11b0764dcdf2b6dc22441.wav,4
3,49883c9646792f3622a518531f8a4aa4.wav,4
4,3509fc87b2af64d5f47213e43a1188ae.wav,3


In [11]:
label_df['wav_path'] = label_df['track'].map(lambda x: DATA_PATH+'/'+x)
label_df.head()

,track,algorithm,wav_path
0,d5142a1e904db7ef51b8071585c46c68.wav,4,./spcup_2022_training/spcup_2022_training_part...
1,0520f2abfaa77784411e0121ba3cb23d.wav,3,./spcup_2022_training/spcup_2022_training_part...
2,64fc1bd819a11b0764dcdf2b6dc22441.wav,4,./spcup_2022_training/spcup_2022_training_part...
3,49883c9646792f3622a518531f8a4aa4.wav,4,./spcup_2022_training/spcup_2022_training_part...
4,3509fc87b2af64d5f47213e43a1188ae.wav,3,./spcup_2022_training/spcup_2022_training_part...


In [12]:
spect_type = 'mel'
def save_and_record(wav_path):

    image_name = wav_path.split('/')[-1].split('.')[0]
    image_path = IMAGE_PATH+'/'+image_name+'.png'
    return image_path

In [13]:
label_df['image_path'] = label_df['wav_path'].map(lambda x: save_and_record(x))
label_df.head()

,track,algorithm,wav_path,image_path
0,d5142a1e904db7ef51b8071585c46c68.wav,4,./spcup_2022_training/spcup_2022_training_part...,./images/d5142a1e904db7ef51b8071585c46c68.png
1,0520f2abfaa77784411e0121ba3cb23d.wav,3,./spcup_2022_training/spcup_2022_training_part...,./images/0520f2abfaa77784411e0121ba3cb23d.png
2,64fc1bd819a11b0764dcdf2b6dc22441.wav,4,./spcup_2022_training/spcup_2022_training_part...,./images/64fc1bd819a11b0764dcdf2b6dc22441.png
3,49883c9646792f3622a518531f8a4aa4.wav,4,./spcup_2022_training/spcup_2022_training_part...,./images/49883c9646792f3622a518531f8a4aa4.png
4,3509fc87b2af64d5f47213e43a1188ae.wav,3,./spcup_2022_training/spcup_2022_training_part...,./images/3509fc87b2af64d5f47213e43a1188ae.png


In [14]:
label_df['track'] = label_df['track'].map(lambda x: x.split('.')[0])
label_df.head()

,track,algorithm,wav_path,image_path
0,d5142a1e904db7ef51b8071585c46c68,4,./spcup_2022_training/spcup_2022_training_part...,./images/d5142a1e904db7ef51b8071585c46c68.png
1,0520f2abfaa77784411e0121ba3cb23d,3,./spcup_2022_training/spcup_2022_training_part...,./images/0520f2abfaa77784411e0121ba3cb23d.png
2,64fc1bd819a11b0764dcdf2b6dc22441,4,./spcup_2022_training/spcup_2022_training_part...,./images/64fc1bd819a11b0764dcdf2b6dc22441.png
3,49883c9646792f3622a518531f8a4aa4,4,./spcup_2022_training/spcup_2022_training_part...,./images/49883c9646792f3622a518531f8a4aa4.png
4,3509fc87b2af64d5f47213e43a1188ae,3,./spcup_2022_training/spcup_2022_training_part...,./images/3509fc87b2af64d5f47213e43a1188ae.png


In [15]:
col_map = {
    'track': 'audio_id',
    'algorithm': 'labels',
    'wav_path': 'wav',
    'image_path': 'image'
}
label_df.rename(col_map, axis=1, inplace=True)
label_df.head()

,audio_id,labels,wav,image
0,d5142a1e904db7ef51b8071585c46c68,4,./spcup_2022_training/spcup_2022_training_part...,./images/d5142a1e904db7ef51b8071585c46c68.png
1,0520f2abfaa77784411e0121ba3cb23d,3,./spcup_2022_training/spcup_2022_training_part...,./images/0520f2abfaa77784411e0121ba3cb23d.png
2,64fc1bd819a11b0764dcdf2b6dc22441,4,./spcup_2022_training/spcup_2022_training_part...,./images/64fc1bd819a11b0764dcdf2b6dc22441.png
3,49883c9646792f3622a518531f8a4aa4,4,./spcup_2022_training/spcup_2022_training_part...,./images/49883c9646792f3622a518531f8a4aa4.png
4,3509fc87b2af64d5f47213e43a1188ae,3,./spcup_2022_training/spcup_2022_training_part...,./images/3509fc87b2af64d5f47213e43a1188ae.png


In [16]:
train_df, test_df = train_test_split(label_df, test_size=0.15)

In [17]:
train_data_dict = {'data': train_df.to_dict('records')}
with open("./final_train_data.json", "w") as outfile:
    json.dump(train_data_dict, outfile)

test_data_dict = {'data': test_df.to_dict('records')}
with open("./final_test_data.json", "w") as outfile:
    json.dump(test_data_dict, outfile)

Model Training

In [8]:
args = argparse.Namespace()

args.data_train = './final_train_data.json'
args.data_val = './final_test_data.json'
args.data_eval = '.json'
args.n_class = 6
args.model = 'ast'
args.dataset = 'speechcommands'
args.exp_dir = '.'
args.lr = 0.001
args.optim = 'adam'
args.batch_size = 12
args.num_workers =32
args.n_epochs = 10
args.lr_patience = 2
args.n_print_steps = 100
args.save_model = None # 
args.freqm = 0
args.timem = 0
args.mixup = 0
args.bal = False
args.fstride = 10
args.tstride = 10
args.imagenet_pretrain = True
args.audioset_pretrain = False


In [9]:
# dataset spectrogram mean and std, used to normalize the input
norm_stats = {'audioset':[-4.2677393, 4.5689974], 'esc50':[-6.6268077, 5.358466], 'speechcommands':[-6.845978, 5.5654526]}
target_length = {'audioset':1024, 'esc50':512, 'speechcommands':128}
# if add noise for data augmentation, only use for speech commands
noise = {'audioset': False, 'esc50': False, 'speechcommands':True}

audio_conf = {'num_mel_bins': 128, 'target_length': target_length[args.dataset], 'freqm': args.freqm, 'timem': args.timem, 'mixup': args.mixup, 'dataset': args.dataset, 'mode':'train', 'mean':norm_stats[args.dataset][0], 'std':norm_stats[args.dataset][1],
                'noise':noise[args.dataset]}
val_audio_conf = {'num_mel_bins': 128, 'target_length': target_length[args.dataset], 'freqm': 0, 'timem': 0, 'mixup': 0, 'dataset': args.dataset, 'mode':'evaluation', 'mean':norm_stats[args.dataset][0], 'std':norm_stats[args.dataset][1], 'noise':False}


In [10]:
if args.bal == 'bal':
    print('balanced sampler is being used')
    samples_weight = np.loadtxt(args.data_train[:-5]+'_weight.csv', delimiter=',')
    sampler = WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)

    train_loader = torch.utils.data.DataLoader(
        dataloader.AudiosetDataset(args.data_train, audio_conf=audio_conf),
        batch_size=args.batch_size, sampler=sampler, num_workers=args.num_workers, pin_memory=True)
else:
    print('balanced sampler is not used')
    train_loader = torch.utils.data.DataLoader(
        dataloader.AudiosetDataset(args.data_train, audio_conf=audio_conf),
        batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    dataloader.AudiosetDataset(args.data_val, audio_conf=val_audio_conf),
    batch_size=args.batch_size*2, shuffle=False, num_workers=args.num_workers, pin_memory=True)

balanced sampler is not used
---------------the train dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process speechcommands
use dataset mean -6.846 and std 5.565 to normalize the input.
now use noise augmentation
number of classes is 6
---------------the evaluation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process speechcommands
use dataset mean -6.846 and std 5.565 to normalize the input.
number of classes is 6


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [11]:
audio_model = ASTModel(label_dim=args.n_class, fstride=args.fstride, tstride=args.tstride, input_fdim=128,
                                input_tdim=target_length[args.dataset], imagenet_pretrain=args.imagenet_pretrain,
                                audioset_pretrain=args.audioset_pretrain, model_size='base384')


---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=144


In [12]:
print('Now starting training for {:d} epochs'.format(args.n_epochs))
train(audio_model, train_loader, val_loader, args)

Now starting training for 10 epochs
running on cuda
Total parameter number is : 86.911 million
Total trainable parameter number is : 86.911 million
scheduler for speech commands is used
now training with speechcommands, main metrics: acc, loss function: BCEWithLogitsLoss(), learning rate scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x7fc016847390>
current #steps=0, #epochs=1
start training...
---------------
2022-03-05 12:41:08.089102
current #epochs=1, #steps=0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [1][100/425]	Per Sample Total Time 0.01625	Per Sample Data Time 0.00293	Per Sample DNN Time 0.01332	Train Loss 0.4271	
Epoch: [1][200/425]	Per Sample Total Time 0.01363	Per Sample Data Time 0.00151	Per Sample DNN Time 0.01212	Train Loss 0.3926	
Epoch: [1][300/425]	Per Sample Total Time 0.01278	Per Sample Data Time 0.00104	Per Sample DNN Time 0.01174	Train Loss 0.3684	
Epoch: [1][400/425]	Per Sample Total Time 0.01233	Per Sample Data Time 0.00079	Per Sample DNN Time 0.01154	Train Loss 0.3449	
start validation
acc: 0.491111
AUC: 0.844741
Avg Precision: 0.396650
Avg Recall: 1.000000
d_prime: 1.434203
train_loss: 0.342345
valid_loss: 0.703310
validation finished
Epoch-1 lr: 0.001
epoch 1 training time: 77.899
---------------
2022-03-05 12:42:25.988273
current #epochs=2, #steps=425


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [2][75/425]	Per Sample Total Time 0.01792	Per Sample Data Time 0.00434	Per Sample DNN Time 0.01358	Train Loss 0.2550	
Epoch: [2][175/425]	Per Sample Total Time 0.01407	Per Sample Data Time 0.00192	Per Sample DNN Time 0.01214	Train Loss 0.2518	
Epoch: [2][275/425]	Per Sample Total Time 0.01301	Per Sample Data Time 0.00125	Per Sample DNN Time 0.01176	Train Loss 0.2506	
Epoch: [2][375/425]	Per Sample Total Time 0.01253	Per Sample Data Time 0.00094	Per Sample DNN Time 0.01159	Train Loss 0.2554	
start validation
acc: 0.583333
AUC: 0.877061
Avg Precision: 0.247364
Avg Recall: 1.000000
d_prime: 1.641084
train_loss: 0.254176
valid_loss: 0.697673
validation finished
Epoch-2 lr: 0.001
epoch 2 training time: 78.766
---------------
2022-03-05 12:43:44.756031
current #epochs=3, #steps=850


...

Epoch: [8][25/425]	Per Sample Total Time 0.03133	Per Sample Data Time 0.01249	Per Sample DNN Time 0.01884	Train Loss 0.1123	
Epoch: [8][125/425]	Per Sample Total Time 0.01539	Per Sample Data Time 0.00263	Per Sample DNN Time 0.01276	Train Loss 0.1212	
Epoch: [8][225/425]	Per Sample Total Time 0.01355	Per Sample Data Time 0.00149	Per Sample DNN Time 0.01206	Train Loss 0.1175	
Epoch: [8][325/425]	Per Sample Total Time 0.01285	Per Sample Data Time 0.00106	Per Sample DNN Time 0.01179	Train Loss 0.1209	
start validation
acc: 0.813333
AUC: 0.967509
Avg Precision: 0.606338
Avg Recall: 1.000000
d_prime: 2.609760
train_loss: 0.123014
valid_loss: 0.664160
validation finished
Epoch-8 lr: 0.00052200625
epoch 8 training time: 79.004
---------------
2022-03-05 12:51:40.654187
current #epochs=9, #steps=3400


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [9][0/425]	Per Sample Total Time 0.46516	Per Sample Data Time 0.30685	Per Sample DNN Time 0.15830	Train Loss 0.1332	
Epoch: [9][100/425]	Per Sample Total Time 0.01659	Per Sample Data Time 0.00310	Per Sample DNN Time 0.01350	Train Loss 0.1632	
Epoch: [9][200/425]	Per Sample Total Time 0.01393	Per Sample Data Time 0.00160	Per Sample DNN Time 0.01233	Train Loss 0.1486	
Epoch: [9][300/425]	Per Sample Total Time 0.01305	Per Sample Data Time 0.00110	Per Sample DNN Time 0.01195	Train Loss 0.1612	
Epoch: [9][400/425]	Per Sample Total Time 0.01261	Per Sample Data Time 0.00084	Per Sample DNN Time 0.01177	Train Loss 0.1770	
start validation
acc: 0.754444
AUC: 0.949218
Avg Precision: 0.525066
Avg Recall: 1.000000
d_prime: 2.315514
train_loss: 0.175586
valid_loss: 0.676457
validation finished
Epoch-9 lr: 0.00044370531249999997
epoch 9 training time: 75.332
---------------
2022-03-05 12:52:55.985616
current #epochs=10, #steps=3825


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [10][75/425]	Per Sample Total Time 0.01825	Per Sample Data Time 0.00361	Per Sample DNN Time 0.01464	Train Loss 0.1542	
Epoch: [10][175/425]	Per Sample Total Time 0.01427	Per Sample Data Time 0.00160	Per Sample DNN Time 0.01267	Train Loss 0.1479	
Epoch: [10][275/425]	Per Sample Total Time 0.01319	Per Sample Data Time 0.00105	Per Sample DNN Time 0.01214	Train Loss 0.1451	
Epoch: [10][375/425]	Per Sample Total Time 0.01267	Per Sample Data Time 0.00079	Per Sample DNN Time 0.01188	Train Loss 0.1456	
start validation
acc: 0.740000
AUC: 0.945889
Avg Precision: 0.541920
Avg Recall: 1.000000
d_prime: 2.271562
train_loss: 0.147380
valid_loss: 0.676821
validation finished
Epoch-10 lr: 0.00037714951562499996
epoch 10 training time: 74.969
